In [14]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [15]:
df_construction = pd.read_csv("data/chantiers-perturbants.csv", sep=";")

In [16]:
df_construction.head()

,Identifiant,Identifiant CTV,Code postal de l'arrondissement,Numéro de STV,Typologie,Maitre d'ouvrage,Objet,Description,Voie(s),Précisions de localisation,Date de début,Date de fin,Impact sur la circulation,Détail de l'impact sur la circulation,Niveau de perturbation,Statut,URL LettreInfoChantier,geo_shape,geo_point_2d
0,CP001735,NaN,75018.0,9.0,1.0,SG Mission des JO 2024,CONSTRUCTION_IMMEUBLE,Arena 2,avenue de la Porte de la Chapelle,n°6,2020-09-07,2023-07-31,BARRAGE_TOTAL,entre le bd Ney et l'avenue de la pte de la Ch...,2.0,2,NaN,"{""coordinates"": [[[2.3603260350936983, 48.8987...","48.89941650246351, 2.3610672014134044"
1,CP002276,NaN,75017.0,9.0,3.0,ICF HABITAT,CONSTRUCTION_IMMEUBLE,Création de 111 logements sociaux.\nEmprise pr...,Voie Bus,Entre le 188 bis et 188 ter avenue de Clichy,2022-05-02,2024-06-30,RESTREINTE,Déviation bus dans circulation générale\nCréat...,2.0,2,http://xdir-CGPub-prd.ressources.paris.mdp/Pdf...,"{""coordinates"": [[[2.315884598767695, 48.89320...","48.89326507701689, 2.3156964416358314"
2,CP002368,654245,75005.0,12.0,2.0,Eau de Paris,ENTRETIEN_RESEAU,Renouvellement conduits EP,"Gobelins, Monge, Censier et Mirbel",13-17 av Gobelins + 111-115 rue Monge + 30 rue...,2022-08-08,2023-01-20,RESTREINTE,Déviation circulation générale (Bus + vélos),2.0,2,NaN,"{""coordinates"": [[[2.352170678013787, 48.84077...","48.84036171643268, 2.3520135239093847"
3,CP002390,689822,75014.0,12.0,3.0,Congrégation St Joseph,REHABILITATION_IMMEUBLE,NaN,Rue Méchain,du 19b au 21,2022-10-05,2023-12-15,RESTREINTE,"Maintien de 4,00m de circulation",2.0,2,NaN,"{""coordinates"": [[[[2.3384636559592034, 48.835...","48.83573836601194, 2.3385684617326348"
4,CP002429,662837,75015.0,13.0,3.0,SCCV Porte de Brancion,CONSTRUCTION_IMMEUBLE,"Construction neuve, résidence sociale et comme...",Avenue de la Porte de Brancion,Carrefour au-dessus du périphérique,2022-11-14,2024-03-15,RESTREINTE,"restriction à é files de circulation, maintien...",2.0,2,NaN,"{""coordinates"": [[[2.299950868215911, 48.82590...","48.82574599992239, 2.300005595088703"


In [26]:
df_construction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Identifiant                            192 non-null    object        
 1   Identifiant CTV                        128 non-null    object        
 2   Code postal de l'arrondissement        190 non-null    float64       
 3   Numéro de STV                          191 non-null    float64       
 4   Typologie                              189 non-null    float64       
 5   Maitre d'ouvrage                       188 non-null    object        
 6   Objet                                  191 non-null    object        
 7   Description                            157 non-null    object        
 8   Voie(s)                                190 non-null    object        
 9   Précisions de localisation             174 non-null    object    

In [18]:
df_bike_counts = pd.read_parquet("data/train.parquet")

In [19]:
df_bike_counts = df_bike_counts[
    ["counter_name", "latitude", "longitude", "date", "log_bike_count"]
]

In [20]:
df_bike_counts.head()

,counter_name,latitude,longitude,date,log_bike_count
48321,28 boulevard Diderot E-O,48.846028,2.375429,2020-09-01 02:00:00,0.000000
48324,28 boulevard Diderot E-O,48.846028,2.375429,2020-09-01 03:00:00,0.693147
48327,28 boulevard Diderot E-O,48.846028,2.375429,2020-09-01 04:00:00,0.000000
48330,28 boulevard Diderot E-O,48.846028,2.375429,2020-09-01 15:00:00,1.609438
48333,28 boulevard Diderot E-O,48.846028,2.375429,2020-09-01 18:00:00,2.302585


In [23]:
df_bike_counts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   counter_name    455163 non-null  category      
 1   latitude        455163 non-null  float64       
 2   longitude       455163 non-null  float64       
 3   date            455163 non-null  datetime64[ns]
 4   log_bike_count  455163 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(3)
memory usage: 17.8 MB


In [22]:
# Convert date columns to datetime objects for easier manipulation
df_construction["Date de début"] = pd.to_datetime(df_construction["Date de début"])
df_construction["Date de fin"] = pd.to_datetime(df_construction["Date de fin"])
df_bike_counts["date"] = pd.to_datetime(df_bike_counts["date"])

In [27]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).kilometers

In [28]:
DISTANCE_THRESHOLD = 1.0  # kilometers

In [30]:
df_construction["geo_point_2d"].head()

0    48.89941650246351, 2.3610672014134044
1    48.89326507701689, 2.3156964416358314
2    48.84036171643268, 2.3520135239093847
3    48.83573836601194, 2.3385684617326348
4     48.82574599992239, 2.300005595088703
Name: geo_point_2d, dtype: object

In [31]:
df_bike_counts.to_csv("counters with latitude and longitude.csv")